In [1]:
## Imports
import torch
import numpy as np
import sklearn
import os, errno
import sys
from datetime import datetime
import time
import csv

import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

from Instseg_model import MultiLayerFastLocalGraphModelV2 as model1
from dataset import pcloader
from graph_generation import gen_multi_level_local_graph_v3

from math import floor, ceil
from scipy.stats import mode

import open3d as o3d
from plot_utils import add_staple_patch
from utils import parse_labelfile


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
## UTILS

## Parse txt files that list scan and label paths per row eg test.txt 
def parse_dataset_file(data_set_filepath):
    """
    Looks at the given text file that list scan and label path per row eg test.txt
    Returns:
     - a list of dicts, where each contains the "scan_path", the "label_path" and the sample "name"
    """
    
    samples = []
    with open(data_set_filepath, 'r') as setfile:
        lines = setfile.readlines()
        for line in lines:
            line = line.strip().strip("[]") # removing the [] bookending each line
            scan, label = line.split(",")
            scan_path = scan.strip().strip("'")
            label_path = label.strip().strip("'")
            name = label_path.split("/")[-1].strip(".txt")
            sample = {"scan_path": scan_path, "label_path": label_path, "name": name}
            samples.append(sample)
    return samples

In [3]:
## CLASSIFICATION INFERENCE

## Hight Level Config Settings

graph_gen_kwargs = {
	'add_rnd3d': True,
	'base_voxel_size': 0.8,
	'downsample_method': 'random',
	'level_configs': [
		{'graph_gen_kwargs': {'num_neighbors': 64, 'radius': 0.4},
		 'graph_gen_method': 'disjointed_rnn_local_graph_v3',
		 'graph_level': 0,
		 'graph_scale': 1},
		{'graph_gen_kwargs': {'num_neighbors': 192, 'radius': 1.2},
		 'graph_gen_method': 'disjointed_rnn_local_graph_v3',
		 'graph_level': 1,
		 'graph_scale': 1}]
}

def configure_model(model_params_path, max_cls_classes=3, max_inst_classes=7, verbose=False):
    a = time.time()
    model = model1(num_classes=max_cls_classes, max_instance_no=max_inst_classes)
    if os.path.isfile(model_params_path):
        model.load_state_dict(torch.load(model_params_path))
    else:
        if verbose:
            print(f"[ModelParamPathError] {model_params_path} does not exist")
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), model_params_path)
    b = time.time()
    if verbose:
        print(f"Model Setup Time (secs) : {b-a}")
    return model

def classify_scan(model, scan_path, verbose=False):
    """
    Returns 
     - the x|y|z|cls|inst for all keypoints in the cloud as a Kx5 array
     - and also as a dict
    """
    a = time.time()
    pointxyz, offset = pcloader(scan_path)
    vertex_coord_list, keypoint_indices_list, edges_list = \
    gen_multi_level_local_graph_v3(pointxyz,0.6,graph_gen_kwargs['level_configs'])
    last_layer_v = vertex_coord_list[-1]

    ## conversions: type precision
    vertex_coord_list = [p.astype(np.float32) for p in vertex_coord_list]
    keypoint_indices_list = [e.astype(np.int32) for e in keypoint_indices_list]
    edges_list = [e.astype(np.int32) for e in edges_list]

    ## conversions: numpy array to tensor
    vertex_coord_list = [torch.from_numpy(item) for item in vertex_coord_list]
    keypoint_indices_list = [torch.from_numpy(item).long() for item in keypoint_indices_list]
    edges_list = [torch.from_numpy(item).long() for item in edges_list]

    ## Run graph through GNN model
    batch = (vertex_coord_list, keypoint_indices_list, edges_list)
    cls_seg, inst_seg = model(*batch)

    ## Filter classification probabilities for the most probable
    cls_preds = torch.argmax(cls_seg, dim=1)
    inst_preds = torch.argmax(inst_seg, dim=1)
    
    ## expand the shape of the array
    cls_preds = np.expand_dims(cls_preds, axis=1)
    inst_preds = np.expand_dims(inst_preds, axis=1)

    b = time.time()
    if verbose:
        print("Scan Inference Time (secs): ", b-a)
        print()
    
    return np.hstack((last_layer_v, cls_preds, inst_preds)),\
            {'vertices': last_layer_v, 'cls_preds': cls_preds, 'inst_preds': inst_preds}

def filter_out_background(scan_data):
    non_bg_idx = ~np.logical_or(scan_data[:, 3] == 0, scan_data[:, 4] == 0)
    non_bg = scan_data[non_bg_idx]
    return non_bg, non_bg_idx

def count_cluster_by_instance_prediction(scan_data, threshold_factor=0.5):
    """
    Returns a cluster_count ie the number of clusters on the cls field/col that has at least a threshold number of members
    Also: a dict of intermediary/final counts, types and thresholds
    """
    inst, inst_count = np.unique(scan_data[:,4], return_counts=True)
    inst_count_threshold = threshold_factor * np.mean(inst_count)
    reduced_idx = np.where(inst_count > inst_count_threshold)
    cluster_count = np.sum(inst_count > inst_count_threshold)
    reduced = inst[reduced_idx]
    
    return cluster_count, {'orig_insts': inst,
                           'orig_inst_ct': inst_count,
                           'inst_ct_thresh': inst_count_threshold,
                           'reduced_insts': reduced}
def draw_labels(welds, ax):
    for weld in welds:
        markers = add_staple_patch(ax ,weld['xloc'], weld['yloc'], weld["yaw"], weld['cls'] )
        
def _plot(path, points, values, title="", caption="", overlay_points=None, labels_dict=None ):
    """
    path - output path
    points - matrix of x, y, etc cols
    values - assigned value per point ie predicted/truth cls/inst
    [optional] overlay_points = [{"xs":[], "ys":[], "cs":_, "marker":_, "label"=_}, ...]
    labels_dict - to overlay the staple patch
    """
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title(title)
    im = ax.scatter(points[:,0], points[:,1],s=0.25,c=values)
    

    if overlay_points:    
        for pts in overlay_points:
            _im = ax.scatter(pts["xs"], pts["ys"], s=pts["ss"], c=pts["cs"], marker=pts["marker"])
    
    if labels_dict:
        draw_labels(labels_dict["welds"], ax)
            
    ax.set_xlabel("x [mm]")
    ax.set_ylabel("y [mm]")

        
    legend_ = ax.legend(*im.legend_elements(), bbox_to_anchor=(1.1, 1), loc="upper right")
    ax.add_artist(legend_)

    ax.text(0.5, -0.5, caption, style='italic', \
        horizontalalignment='center', verticalalignment='top', transform=ax.transAxes)
    axes=plt.gca()
    axes.set_aspect(1)
    if path:
        plt.savefig(path, dpi=150)
        plt.close()
        plt.cla()
        plt.clf()
    else:
        plt.show
    
def save_prediction_plots(non_bg_matrix, labels_dict=None, cls_tag=None, inst_tag=None, cls_col=3, inst_col=4, inst_seg_dir_path="./plots/inst/", cls_seg_dir_path="./plots/cls/", verbose=False):
    """
    non_bg_matrix -- x|y|z|cls|inst
    // tag -- eg A_xycls_eps0_45_50
    tag -- eg A_xycls
    """
    
    ## [RED FLAG] - what if dir_path comes through as None
    f_tag = "[save_prediction_plots]"
    f_msg = []
    
    if not (cls_tag and inst_tag):
        print("[ERROR] No specfied cls_tag or inst_tag args. Plots not generated")
    else:
        if cls_tag:
            cls_output_path = cls_seg_dir_path+cls_tag+".png" if cls_seg_dir_path else None
            _plot(cls_output_path, non_bg_matrix[:, :2], non_bg_matrix[:, cls_col], title=cls_tag, labels_dict=labels_dict)
            f_msg.append(cls_output_path)

        if inst_tag:
            inst_output_path = inst_seg_dir_path+inst_tag+".png" if inst_seg_dir_path else None
            _plot(inst_output_path, non_bg_matrix[:, :2], non_bg_matrix[:, inst_col], title=inst_tag, labels_dict=labels_dict)
            f_msg.append(inst_output_path)

        if verbose:
            print(f"{f_tag}: Done --> {f_msg}")
            


###################################################
###  EVENTUALLY BUT SKIPPING THIS FOR NOW #########
## Need to record the model weights version
## Need to record the number of instances identified, the instance_seg_loss and cls_loss
def save_prediction_stats(model, scan_path, output_file):
    """
    Intialize text file if non-existent with name | predictions_clusters | prediction cluster counts | cluster_count_threshold | reduced_clusters 
    """
    pass

###################################################

def run_preclustering(model, scan_path, labels_dict=None, sample_tag="", inst_seg_img_dir_path="./plots/inst_seg/", cls_seg_img_dir_path="./plots/cls_seg/"):
    """
    Runs the scan through the model
    Filters out the background predictions
    Plots cls and inst predictions post filtering and saves plots to file
    Returns the Nx5 data of non-background points --> x|y|z|cls|inst
    """
    # KX5 array --> x|y|z|cls|inst
    scan_data, _ = classify_scan(model, scan_path, verbose=True)
    
    # non bg NX5 array --> x|y|z|cls|inst
    scan_data, _ = filter_out_background(scan_data)
    
    ## [RED FLAG] - No use of the cluster_count or data
    _, cluster_data = count_cluster_by_instance_prediction(scan_data)
    
    ## [RED FLAG] what if dir_path specified as None. Need to catch this cas
#     save_prediction_plots(scan_data, cls_tag=sample_tag, inst_tag=sample_tag, cls_seg_dir_path=cls_seg_img_dir_path, inst_seg_dir_path=inst_seg_img_dir_path, labels_dict=labels_dict)
    return scan_data, {"0_scan": scan_path, "0_tag":sample_tag}

In [4]:
## CLUSTERING AND CLEAN UP

def scale_xyz(scan_data):
    ## Scaling the numerical data columns ie x, y, z cols, then appending the orig cols for return
    
    data_numerical = scan_data[:, :3]
    X = StandardScaler()
    scaled_num = X.fit_transform(data_numerical)
    return np.hstack((scaled_num, scan_data))


def save_cluster_plots(non_bg_matrix, values, title=None, dir_path="./plots/", verbose=False):
    """
   ### non_bg_matrix -- scaled_x|scaled_y|scaled_z|x|y|z|cls|inst
    non_bg_matrix -- x|y|z|cls|inst
    values -- cluster id
    tag -- eg A_x_eps0.05_minpts30
    """
    f_tag = "[save_cluster_plots]"
    f_msg = ""
    
    if not title:
        print("[ERROR] No specfied title args. Plot not generated")
    else:
        cls_output_path = dir_path+title+".png"
        _plot(cls_output_path, non_bg_matrix[:, :2], values, title=title)
        f_msg = cls_output_path
        
        if verbose:
            print(f"{f_tag}: Done --> {f_msg}")

def processDBSCAN2(data, sample_tag, cols_tag, eps, min_samples, verbose=False):
    """
    data  -- eg x|y
    
    """
    
    title = f"{sample_tag}_{cols_tag}_eps{eps}_min{min_samples}"
    if verbose:
        print("title:", title )
    DBSCAN_model = DBSCAN(eps=eps, min_samples=min_samples)
    DBSCAN_result = DBSCAN_model.fit_predict(data)
    DBSCAN_clusters, DBSCAN_cluster_counts = np.unique(DBSCAN_result, return_counts=True)

    if verbose:
        print("DBSCAN_clusters")
        print(DBSCAN_clusters)
        print()

        print("DBSCAN_cluster_counts")
        print(DBSCAN_cluster_counts)
        print()
    
    n_rows, n_cols = data.shape
    out = np.zeros((n_rows, 1))
    for cluster in DBSCAN_clusters:
        idx = np.where(DBSCAN_result == cluster)
        out[idx] = cluster

    return out, title, {"cluster_vals": DBSCAN_clusters, "cluster_counts": DBSCAN_cluster_counts}


def run_DBSCAN_clustering_2D(scan_data, eps, min_samples, sample_tag="", dir_path="./plots/"):
    out, title, _ = processDBSCAN2(scan_data[:, :2], sample_tag, "xy", eps, min_samples, verbose=False)
    save_cluster_plots(scan_data, out, title=title, dir_path=dir_path )
    return np.hstack((scan_data, out))


def postcluster_grouping(scan_data, clusters=None):
    
    meta = {}
    
    shortlist_cluster_idx = clusters["cluster_ids"] if clusters else None
    if shortlist_cluster_idx:
        meta["2_count_pre_reduction"] = sum(clusters["cluster_counts"])
    
    else:
        ## Cluster Size Check: Setting cluster count bounds to weed out noise cluster and merged clusters
        expected_cluster_count, _ = count_cluster_by_instance_prediction(scan_data)
        expected_mean = scan_data.shape[0]/expected_cluster_count
        count_range = (0.5*expected_mean, 1.5*expected_mean)
        meta["1_orig_nonbg_count"] = scan_data.shape[0]
        meta["4_clustering_size_range"] = count_range

        ## Cluster Size Check: Removing the noise and merged clusters
        clusters, cluster_counts = np.unique(scan_data[:, -1], return_counts=True)
        filtered_clusters = [] 
        for c_id, c_count in zip(clusters, cluster_counts):
            if c_id != -1:
                if c_count >= count_range[0] and c_count <= count_range[1]:
                    filtered_clusters.append((int(c_id), int(c_count)))

        ## [ERROR] should be storing a copy
        meta["5_filtered_cluster_presort"] = filtered_clusters

        ## Cluster Size Check: Sorting into descending order of cluster size
        filtered_clusters.sort(reverse=True, key = lambda e: e[1])

        ## [ERROR] should be storing a copy
        meta["6_filtered_cluster_sorted"] = filtered_clusters
        meta["2_filtered_by_size_count"] = sum([c_count for c_id, c_count in filtered_clusters])

        ## Cluster Size Check: Returning only expected_cluster_count or less number of clusters
        ## Effectively relying on GNN instance predictions for number of welds detected, at least
        if len(filtered_clusters) > expected_cluster_count:
            filtered_clusters = filtered_clusters[:expected_cluster_count]
        meta["7_filtered_cluster_final"] = filtered_clusters    

        shortlist_cluster_idx = [c_id for c_id, c_count in filtered_clusters]
    
    ## Reconsituting scan_data without noise, merged or extra clusters; possible size reduction
    scan_data_idx = np.isin(scan_data[:,-1], shortlist_cluster_idx)
    scan_data = scan_data[scan_data_idx]
    meta["3_final_count"] = scan_data.shape[0]
    
    return scan_data, meta

def postcluster_voting(scan_data):

    ## remove the -1 clusters
    scan_data = scan_data[scan_data[:, -1] >= 0]
    
    ## create new voting cols: cls and inst
    n_rows, _ = scan_data.shape
    scan_data = np.hstack((scan_data, np.zeros((n_rows, 1)), np.zeros((n_rows, 1)))) 

    ## expecting col5 to be the clustering result
    clusters = np.unique(scan_data[:, 5])
    for c_id in clusters:
        scan_data_subset_idx = np.where(scan_data[:, 5] == c_id)
        
        ## majority votes
        cls_vote = mode(np.ravel(scan_data[scan_data_subset_idx,3]), keepdims=False)[0]
        inst_vote = mode(np.ravel(scan_data[scan_data_subset_idx,4]), keepdims=False)[0]
        
        ## push to data matrix
        scan_data[scan_data_subset_idx, 6] = cls_vote
        scan_data[scan_data_subset_idx, 7] = inst_vote

    return scan_data

def run_postclustering(scan_data, sample_tag="", dir_path="./plots/", cls_seg_dir_path="./plots/",  inst_seg_dir_path="./plots/", clusters=None ):
    scan_data, meta1 = postcluster_grouping(scan_data, clusters=None) ## ncols remains 6, nrows possibly reduced
    scan_data = postcluster_voting(scan_data)  ## n_cols increased from 6 to 8
    save_prediction_plots(scan_data, cls_tag=f"{sample_tag}_xycls_clean", inst_tag=f"{sample_tag}_xyinst_clean", cls_col=6, inst_col=7, cls_seg_dir_path=cls_seg_dir_path, inst_seg_dir_path=inst_seg_dir_path)
    return scan_data, meta1

In [5]:
# ## Run clustering on xy(not scaled) data for each scan and plotting results

# ## Model Instantiating
# model_params_path = "./_model/train1-fix3/2023_06_22_17_01_13/params_epoch497_for_min_test_loss.pt"
# model = configure_model(model_params_path, verbose=True)

# ## Looping through the test samples to generate and save plots post DBSCAN cleanup
# test_txt = "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/_data/test.txt"
# test_samples = parse_dataset_file(test_txt)

# clustering_img_dir = "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/_img_dbscan/test_set/"
# trash_dir =  "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/_img_dbscan/test_set/trash/"
# inst_seg_img_directory = "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/_img_instance_seg/test_set/test/"
# cls_seg_img_directory = "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/_img_class_seg/test_set/test/"

# for sample in test_samples[:4]:
#     scan_path = sample["scan_path"]
#     sample_name = sample["name"]
#     scan_data, _ = run_preclustering(model, scan_path,sample_tag="testingDBSCAN", inst_seg_img_dir_path=trash_dir, cls_seg_img_dir_path=trash_dir  )

# #     scan_data = scale_xyz(scan_data)
#     scan_data = run_DBSCAN_clustering_2D(scan_data, 2, 20, sample_tag=f"{sample_name}_", dir_path=clustering_img_dir)
#     scan_data, meta1 = run_postclustering(scan_data, sample_tag=sample_name, dir_path=clustering_img_dir, inst_seg_dir_path=inst_seg_img_directory, cls_seg_dir_path=cls_seg_img_directory)

In [6]:
## Extending IPYNB script to generate weld instances and labels
## naming convention: WELD_1.npy, WELD_1.txt

locationing_dataset_dir = "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/locationing_dataset/" 
label_subdir = "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/locationing_dataset/labels/" 
pc_subdir = "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/locationing_dataset/point_clouds/"

In [7]:
## WELD EXTRACTION AND FILE GENERATION UTILS
## IE DATASET GENERATION

def extract_welds_from_cleaned_segmentation(scan_data, inst_col=7):
    """
    returns a list of welds' xy point array 
    """
    welds=[]
    
    ## looking at the inst col for grouping
    point_inst = scan_data[:, inst_col]    
    insts, insts_counts = np.unique(point_inst, return_counts=True)

    ## collect groupings into a list of xy points
    for inst in insts:
        weld_xy = scan_data[point_inst==inst, :2]
        welds.append(weld_xy)
        
    return welds

def extract_labels_from_labelfile(labelfile_path):
    """
    returns a list of (x) point array 
    """
    label_dict = parse_labelfile(labelfile_path)
    welds = [(weld["xloc"], weld["yloc"], weld["cls"]) for weld in label_dict["welds"]]
    return welds

def match_welds_and_centers(welds, labels):
    
    '''
    returns a list of dicts each with "points_xy" and "xy" keys
    '''
    
    pairings = []
    assert(len(welds) <= len(labels))
    
    ## looping through each grouping of points
    for weld in welds:
        
        ## checking grouping for the limits in x
        ## labels for if within limits in x
        xmin = np.min(weld[:, 0])
        xmax = np.max(weld[:, 0])
#         xy = []
        
        ## missing check for remaining 
        for idx, (x,y, cls) in enumerate(labels):
            if xmin<=x<=xmax:
                xy = labels.pop(idx)
        
        ## pairing points and location label
        pairings.append({"points_xy": weld, "xy":xy, "cls": cls})
        
    return pairings

def create_weld_and_label_files(points_xyinst, labels, filename_prefix="", dir_path="./dataset/", point_cloud_sub="point_clouds/", labels_sub="labels/"):
    if not os.path.isdir(dir_path):
        os.mkdir(directory)
        print(f"created: {dir_path}")
        
    weld_scan_path = f"{dir_path}{point_cloud_sub}"
    if not os.path.isdir(weld_scan_path):
        os.mkdir(weld_scan_path)
        print(f"created: {weld_scan_path}")
        
    weld_label_path = f"{dir_path}{labels_sub}"
    if not os.path.isdir(weld_label_path):
        os.mkdir(weld_label_path)
        print(f"created: {weld_label_path}")
        
    last_file_tag = 0
    scan_filenames = os.listdir(weld_scan_path)
    scan_filenames = [file.split(".")[0] for file in scan_filenames]
    scan_suffixes = [file.split("_")[-1] for file in scan_filenames if file.startswith(filename_prefix)]
#     print("scan_suffixes")
#     print(scan_suffixes)
    
    if len(scan_suffixes) > 0:
        scan_suffixes = [int(suffix) for suffix in scan_suffixes]
        last_file_tag = max(scan_suffixes)
#         print("last_file_tag")
#         print(last_file_tag)
     
    welds = extract_welds_from_cleaned_segmentation(points_xyinst)
#     print("welds: ", len(welds) )
#     print("labels: ", labels )
    
    welds_and_labels = match_welds_and_centers(welds, labels)
    for w_l in welds_and_labels:
        last_file_tag+=1
        np.save(f"{weld_scan_path}{filename_prefix}{last_file_tag}", w_l["points_xy"])
        np.savetxt(f"{weld_label_path}{filename_prefix}{last_file_tag}.txt", np.array(w_l["xy"]).reshape((1,3)),"%1.2f %1.2f %1d" )
#         print(f"created scan: {weld_scan_path}{filename_prefix}{last_file_tag}")
#         print(f"created label: {weld_label_path}{filename_prefix}{last_file_tag}")

In [11]:
def runItemizing(model_params_path, dataset_txt, clustering_img_dir, trash_dir, inst_seg_img_directory, cls_seg_img_directory):

    ## Model Instantiating
    model = configure_model(model_params_path, verbose=True)

    ## Looping through the samples list to generate and save plots
    samples = parse_dataset_file(dataset_txt)
    print("# samples in set: ", len(samples))

    for idx, sample in enumerate(samples):
        scan_path = sample["scan_path"]
        label_path = sample["label_path"]
        sample_name = sample["name"]

        scan_data, _ = run_preclustering(model, scan_path, sample_tag="testingDBSCAN", inst_seg_img_dir_path=trash_dir, cls_seg_img_dir_path=trash_dir  )
        scan_data = run_DBSCAN_clustering_2D(scan_data, 2, 20, sample_tag=f"{sample_name}_", dir_path=clustering_img_dir)
        scan_data, meta1 = run_postclustering(scan_data, sample_tag=sample_name, dir_path=clustering_img_dir, inst_seg_dir_path=inst_seg_img_directory, cls_seg_dir_path=cls_seg_img_directory)

        print(idx, "label_path", label_path)
        labels = extract_labels_from_labelfile(label_path)
        create_weld_and_label_files(scan_data, labels, filename_prefix=f"WELD_{sample_name}_", dir_path=locationing_dataset_dir, point_cloud_sub="point_clouds/", labels_sub="labels/")

In [9]:
#################################################
## Itemizing Welds from the Training Set
#################################################

# ## settings
# model_params_path = "./_model/train1-fix3/2023_06_22_17_01_13/params_epoch497_for_min_test_loss.pt"
# dataset_txt = "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/_data/train.txt"

# root_dir = "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/"
# clustering_img_dir = root_dir+ "_img_dbscan/train_set/"
# trash_dir =  root_dir+ "/_img_dbscan/train_set/trash/"
# inst_seg_img_directory = root_dir+ "_img_instance_seg/train_set/"
# cls_seg_img_directory = root_dir+ "_img_class_seg/train_set/"

# locationing_dataset_dir = root_dir+"locationing_dataset/train/"
# runItemizing(model_params_path, dataset_txt, clustering_img_dir, trash_dir, inst_seg_img_directory, cls_seg_img_directory)

In [12]:
#################################################
## Itemizing Welds from the Test Set
#################################################

## settings
model_params_path = "./_model/train1-fix3/2023_06_22_17_01_13/params_epoch497_for_min_test_loss.pt"
dataset_txt = "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/_data/test.txt"

root_dir = "C:/Users/KZTYLF/Documents/playground/GNN UIs/GNN InstanceSegmentation/Recreating Dataset/"
clustering_img_dir = root_dir+ "_img_dbscan/test_set/"
trash_dir =  root_dir+ "/_img_dbscan/test_set/trash/"
inst_seg_img_directory = root_dir+ "_img_instance_seg/test_set/"
cls_seg_img_directory = root_dir+ "_img_class_seg/test_set/"

locationing_dataset_dir = root_dir+"locationing_dataset/test/" 
runItemizing(model_params_path, dataset_txt, clustering_img_dir, trash_dir, inst_seg_img_directory, cls_seg_img_directory)

Model Setup Time (secs) : 0.07054829597473145
# samples in set:  121
Scan Inference Time (secs):  10.323866605758667

0 label_path ./_data/labels/RH-5-231201627-Pass-2023_06_13-9-19-08-357.txt
Scan Inference Time (secs):  10.128194332122803

1 label_path ./_data/labels/LH-5-231201608-Pass-2023_06_12-10-56-33-911.txt
Scan Inference Time (secs):  9.975475549697876

2 label_path ./_data/labels/LH-5-231201605-Pass-2023_06_12-10-46-15-717.txt
Scan Inference Time (secs):  9.687974691390991

3 label_path ./_data/labels/LH-5-231201596-Pass-2023_06_12-9-22-54-270.txt
Scan Inference Time (secs):  9.312412738800049

4 label_path ./_data/labels/RH-10-231201591-Pass-2023_06_13-11-42-25-737.txt
Scan Inference Time (secs):  10.10603952407837

5 label_path ./_data/labels/RH-7-231201627-Fail-2023_06_13-9-19-17-681.txt
Scan Inference Time (secs):  9.6724374294281

6 label_path ./_data/labels/RH-9-231201590-Pass-2023_06_13-11-39-05-753.txt
Scan Inference Time (secs):  9.764238357543945

7 label_path ./_d

65 label_path ./_data/labels/RH-4-231201590-Fail-2023_06_09-10-48-34-421.txt
Scan Inference Time (secs):  10.473971128463745

66 label_path ./_data/labels/RH-8-231201585-Pass-2023_06_09-9-22-49-024.txt
Scan Inference Time (secs):  10.86557126045227

67 label_path ./_data/labels/RH-5-231201587-Fail-2023_06_09-10-42-44-797.txt
Scan Inference Time (secs):  9.890031814575195

68 label_path ./_data/labels/RH-6-231201625-Pass-2023_06_12-14-17-48-219.txt
Scan Inference Time (secs):  9.78389024734497

69 label_path ./_data/labels/RH-8-231201624-Pass-2023_06_12-14-14-41-990.txt
Scan Inference Time (secs):  9.684354543685913

70 label_path ./_data/labels/RH-10-231201625-Pass-2023_06_12-14-18-06-818.txt
Scan Inference Time (secs):  9.738842010498047

71 label_path ./_data/labels/RH-9-231201583-Fail-2023_06_09-9-17-11-772.txt
Scan Inference Time (secs):  10.548301219940186

72 label_path ./_data/labels/RH-6-231201250-Pass-2023_06_09-9-09-12-098.txt
Scan Inference Time (secs):  9.803160190582275

7

<Figure size 640x480 with 0 Axes>

In [14]:
from throwaway import repeat, repeat_print

repeat()

ImportError: cannot import name 'repeat' from 'throwaway' (C:\Users\KZTYLF\Documents\playground\GNN UIs\GNN InstanceSegmentation\throwaway.py)